# 강의 레퍼런스 : [Python Flask Web Framework_생활코딩](https://youtube.com/playlist?list=PLuHgQVnccGMClNOIuT3b3M4YZjxmult2y)

## 3. 플라스크를 사용하는 이유  

```python
from flask import Flask
import random

app = Flask(__name__)

@app.route('/')
def index():
    return f'random : <strong>{str(random.random())}</strong>' 
    # return 뒤에는 str 변수가 와야 함
    # html 태그를 이용할 수 있음

app.run(port=5001, debug=True)
# port 아규먼트를 이용해서 열릴 포트를 지정 가능함
# debug를 이용해서 변경사항을 F5를 눌러 즉각적으로 확인가능함
# (실제 릴리즈시에는 사용하면 안됨)
```

브라우저 입장에서는 HTML만 알면 됨.  
그러므로 Flask에서 동적으로 HTML을 만들어서 리턴하면 동적으로 페이지 생성가능함.  

## 4. 라우팅

"웹 프레임워크가 라우팅을 어떻게하나?"  
알아도 50%는 파악하는 정도로 중요한 부분임.  
  
하나의 리턴을 포함하는 "함수"가 각 "주소"를 담당함.  
"함수"와 "주소"를 연결하는 것이 라우팅.  
라우팅을 해주는것이 라우터.  
  
동적인 페이지를 생성하기 위해서는 <code><>(꺽쇠)</code>를 이용함.  
①꺽쇠로 페이지 주소로 사용할 변수를 선언.  
②리턴할 함수의 파라미터로 코드를 구성하면 됨.  

```python
# ex) your_port/read/1 로 들어오면 "this is page No.1" 이 웹페이지에 print
@app.route('/read/<page>')
def read(page): # page를 파라미터로 함수 구성
    return f'this is page No.{page}'
```


## 5. 홈페이지 구현

아래와 같은 코드가 있다는 상태에서 설명이 시작됨

```python
from flask import Flask
import random

app = Flask(__name__)

@app.route('/')
def index():
    return '''<!doctype html>
    <html>
        <body>
            <h1><a href="/">WEB</a></h1>
            <ol>
                <li><a href="/read/1/">html</a></li>
                <li><a href="/read/2/">css</a></li>
                <li><a href="/read/3/">javascript</a></li>
            </ol>
            <h2>Welcome</h2>
            Hello, Web
        </body>
    </html>
    '''

# ex
@app.route('/read/<page>')
def read(page):
    return f'this is page No.{page}'

app.run(port=5001, debug=True)
```

이 코드에서 반복되는 부분(=동적인 부분?)을 파이썬의 딕셔너리 구조체로 변경이 가능함.  
```python
<li><a href="/read/1/">html</a></li>
<li><a href="/read/2/">css</a></li>
<li><a href="/read/3/">javascript</a></li>
```

아래와 같이 </code>for</code>와 <code>dict</code>를 이용해서 딕셔너리를  
string으로 변환해서 대체하면 조금 더 fancy하게 만들 수 있음.  
\# data type을 무엇으로 사용했는지를 잘 기억해둘 필요가 있어 보임  
\# \<a ref\='세부주소'>를 이용해서 링크와 연결하는 문법을 기억해둘 필요가 있어 보임

```python
topics = [
    {'id':1, 'title':'html', 'body':'html is ...'},
    {'id':2, 'title':'css', 'body':'css is ...'},
    {'id':3, 'title':'javascript', 'body':'javascript is ...'}
]

@app.route('/')
def index():
    liTags = '' # f-string으로 만들 문장
    for topic in topics:
        liTags = liTags + f'<li><a href="/read/{topic["id"]}">{topic["title"]}</a></li>' # <a href="sub주소">를 이용
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">WEB</a></h1>
            <ol>
                {liTags}
            </ol>
            <h2>Welcome</h2>
            Hello, Web
        </body>
    </html>
    '''
```

일반적으로 topics는 <code>DB</code>를 이용해서 구성함.  

## 6. 읽기

기억해둬야 할 것은 다음과 같음  
1. 반복된 string은 템플릿(template)구현으로 해결  
2. int가 들어오는 경우 데코레이터에 hint를 명시  
이것을 이용해서 전체 코드를 편집하면 다음과 같음

```python
from flask import Flask
import random

app = Flask(__name__)

topics = [
    {'id':1, 'title':'html', 'body':'html is ...'},
    {'id':2, 'title':'css', 'body':'css is ...'},
    {'id':3, 'title':'javascript', 'body':'javascript is ...'}
]


# 반복되는 부분을 템플릿(html 생성기)으로 처리, 지금은 임시로 함수로 구현
def template(contents, content):
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">WEB</a></h1>
            <ol>
                {contents}
            </ol>
            {content}
            Hello, Web
        </body>
    </html>
    '''

# 모두가 동일한 contents를 리턴하는 함수 구현
def getContents():
    liTags = '' # f-string으로 만들 문장
    for topic in topics:
        liTags = liTags + f'<li><a href="/read/{topic["id"]}">{topic["title"]}</a></li>'
    return liTags

@app.route('/')
def index():
    return template(getContents(), '<h2>Welcome</h2>Hello, WEB')

# ex
@app.route('/read/<int:id>')
def read(id): # 여기에서 id로 들어오는 내용은 string임 
              # 따라서 뒤의 == 문법을 이용하기 까다로움 
              # 그러므로 데코레이터에 int라는 hint를 지정해서 정수로 형변환을 해줌
    for topic in topics:
        if id == topic['id']:
            title = topic['title']
            body = topic['body']
    # print(title, body)
    return template(getContents(), f'<h2>{title}</h2>{body}')

app.run(port=5001, debug=True)
```

## 7. 쓰기

구현 코드 설계  
1. 제일 처음 templete안의 content안에서 버튼을 누르면 create링크를 달고 /create/와 연결되도록 설계
2. 처음 진입은 post로 들어오고, create 버튼을 누르고 진입하면 get으로 진입.  
3. create버튼을 누를 경우 db에 title과 body의 내용이 들어가도록 할 것.  


첫번째,  
- create url로 들어오면 "title", "body"가 placeholder로 구성된 텍스트 박스 2개로 생성되도록 content를 구성  
  
```python
content = '''
            <form action="/create/" method="POST">
                <p><input name="title" type="text" placeholder="title"></p>
                <p><textarea name="body" placeholder="body"></textarea></p>
                <p><input type="submit" value="create"></p>
            </form>
        '''

```

두번째,  
- html에서 text box의 이름을 지정하는 것이 name 테그임.  
- text box에 name을 지정  
- 동작의 트리거를 지정하기 위해서 입력을할수 있는 submit 버튼을 생성.  
- submit박스를 만들기 위해서는 type="submit"을 사용.  

```html
<form action="/create/" method="POST">
    <p><input name="title" type="text" placeholder="title"></p>
    <p><textarea name="body" placeholder="body"></textarea></p>
    <p><input type="submit" value="create"></p>
</form>
```

세번째,  
- 버튼을 클릭할 때 "서버에 값을 전달"하기 위해서 form 태그를 오버랩 해야함.  
- 예를 들어, action이 /asdf/, title이 hello, body가 world라고 입력이 되어있는 상태를 가정  
- 클릭하면 http://127.0.0.1:5001/asdf?title=hello&body=world 로 이동함
- 이러한 url을 통해서 데이터를 전달하는 방식을 GET방식이라고 명명함
- GET방식은 일반적으로 값을 읽어올때(read) 사용함  
- 예를들어 검색엔진의 쿼리는 GET방식으로 데이터 전달함

네번째,  
* 서버에 값을 기록할때는 POST 방식을 사용하는 것이 일반적임.  
* 이를 사용하기 위해서는 form 태그의 method를 사용함.  
* method="POST"를 사용하면 URL로 데이터를 전달하지 않음.  
* 전달된 데이터는 웹 브라우저의 Payload에서 확인할 수 있음.
* POST는 GET 방식보다 더 큰 데이터를 안전하게 전달해줄 수 있음.

```html
<form action="/create/" method="POST">
    ...
</form>
```

다섯번째,  
* 하지만, POST 방식을 사용하면 Method Not Allowd가 뜰 것임
* 왜냐하면, 다시 접속한 /create/는 디폴트로 GET방식만 READ할수 있도록 구현되어 있기 때문.
* 이것을 고려하여 두가지를 더 추가해야함.  
* ① 데코레이터에 <code>method = ['GET', 'POST']</code>를 추가
* ② request 모듈을 임포트해서 호출 방식에 따라 분기문 구현

```python
@app.route('/create/', methods=['GET', 'POST']) # ⭐
def create():
    if request.method == 'GET': # ⭐
        content = '''abc'''
        return template(getContents(), content)
    elif request.method == 'POST': # ⭐
        return redirect(url)
```

여섯번째,  
* POST 방식으로 호출된 이후 form을 꺼내와야 할 필요가 있음
* 그런경우, request.from['your_key']로 가져올 수 있음

```python
title = request.form['title'] # ⭐
body = request.form['body'] # ⭐
```

일곱번째,  
* DB로 가정했던 list에 값을 append하는 로직을 추가함  
* return에 <code>redirect()</code>를 이용해서 새로운 URL로 들어가는 로직을 추가함

```python
    elif request.method == 'POST':
        ...
        return redirect(url) # ⭐
```

## 8. 수정

첫번째  
* update는 create와 거의 비슷한 기능을 수행, 추가적으로 read 기능이 필요함.  
* 따라서, <code>create()</code>를 복사해서 만드는 것이 가장 효율적임
* 변경부분은, /create/를 /update/id/로 바꾸고, 파라미터 추가

```python
@app.route('/update/<int:id>/', methods=['GET', 'POST'])
def update(id):
    ...
    return ...
```

두번째  
* 내용을 표시하기 위해서 GET으로 호출된 경우 ID의 타이틀과 BODY를 가져온다.
* input과 textarea에 title과 body를 할당한다
* action을 update로 바꾸고 submit의 text박스 value에 create를 update로 갱신한다

```python
if request.method == 'GET':
    title = ''
    body = ''
    for topic in topics:
        if id == topic['id']:
            title = topic['title']
            body = topic['body']
            break
    content = f'''
        <form action="/update/{id}/" method="POST">
            <p><input name="title" type="text" placeholder="title" value="{title}"></p>
            <p><textarea name="body" placeholder="body" value="{body}">{body}</textarea></p>
            <p><input type="submit" value="update"></p>
        </form>
    '''
    return template(getContents(), content)

```

세번째  
* POST의 경우 DB에서 title과 body를 읽어서 /read/ URL로 리다이렉트 한다

```python
elif request.method == 'POST':
    title = request.form['title']
    body = request.form['body']
    for topic in topics:
        if id == topic['id']:
            topic['title'] = title
            topic['body'] = body
            break
    url = '/read/' + str(id) + '/'
    return redirect(url)

```

## 8. 수정

첫번째,  
* 템플릿에 delete 버튼을 생성
* delete 버튼은 /delete/{id}/ post방식으로 url 연결
* delete는 POST방식으로 구현해야함

```python
def template(contents, content, id=None): # id 파라미터 추가
    contextUI = '' # 디폴트로 아무 내용이 없는 UI
    
    if id != None: # 만약 id 파라미터가 있다면 업데이트 링크 생성
        contextUI = f'''
            <li><a href="/update/{id}/">update</a></li>
            <li><form action="/delete/{id}/" method="POST"><input type="submit" value="delete"></form></li>
            <!-- 템플릿에 델리트 버튼 생성 -->
        '''
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">WEB</a></h1>
            <ol>
                {contents}
            </ol>
            {content}
            <ul>
                <li><a href = "/create/">create</a></li> 
                {contextUI} <!-- 만약 파라미터 전달되면 update link 생성 -->
            </ul>
            <!-- a href: 링크연결을 하기 위한 부분 -->
            <!-- li: 목록 -->
            <!-- ul: 순서 없는 목록임을 명시 -->
        </body>
    </html>
    '''
```

두번째,  
* 포스트 방식으로 들어올때만 하도록 데코레이터와 함수를 만듬.  
* DB에서 해당 id를 remove하고 redirect할때 root로 가도록 구현.  

```python
@app.route('/delete/<int:id>/', methods=['POST'])
def delete(id):
    for topic in topics:
        if id == topic['id']:
            topics.remove(topic)
            break
    return redirect('/')
```